In [13]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [26]:
df = pd.read_csv('plate9290_RVs.csv').drop(['Unnamed: 0', 'Plate ID'], axis=1)

In [27]:
df

,Target ID,MJD,OBSVHELIO (km/s)
0,apo25m.5226.150-08-RV.2M03252400+4614203,57706,-86.565704
1,apo25m.5226.150-08-RV.2M03252400+4614203,57732,-86.383430
2,apo25m.5226.150-08-RV.2M03252400+4614203,57734,-86.386870
3,apo25m.5226.150-08-RV.2M03252400+4614203,57735,-86.532840
4,apo25m.5226.150-08-RV.2M03252400+4614203,57760,-86.366640
...,...,...,...
4181,apo25m.5226.150-08-RV.2M03415658+4626067,58068,-26.490055
4182,apo25m.5226.150-08-RV.2M03415658+4626067,58085,-26.595278
4183,apo25m.5226.150-08-RV.2M03415658+4626067,58087,-26.603954
4184,apo25m.5226.150-08-RV.2M03415658+4626067,58114,-26.599243


In [28]:
features = ['MJD', 'OBSVHELIO (km/s)']

In [29]:
x = df.loc[:, features].values

In [30]:
y = df.loc[:, ['Target ID']].values

In [31]:
x = StandardScaler().fit_transform(x)

In [32]:
from sklearn.decomposition import PCA

In [33]:
pca = PCA(n_components = 2)

In [34]:
principalComponents = pca.fit_transform(x)

In [35]:
principalDf = pd.DataFrame(data = principalComponents, 
                           columns = ['principal component 1', 'principal component 2'])

In [36]:
finalDf = pd.concat([principalDf, df[['Target ID']]], axis = 1)

In [37]:
finalDf

,principal component 1,principal component 2,Target ID
0,0.928584,-0.999609,apo25m.5226.150-08-RV.2M03252400+4614203
1,0.816866,-0.887886,apo25m.5226.150-08-RV.2M03252400+4614203
2,0.808272,-0.879292,apo25m.5226.150-08-RV.2M03252400+4614203
3,0.803973,-0.874997,apo25m.5226.150-08-RV.2M03252400+4614203
4,0.696552,-0.767571,apo25m.5226.150-08-RV.2M03252400+4614203
...,...,...,...
4181,-0.626040,0.556755,apo25m.5226.150-08-RV.2M03415658+4626067
4182,-0.699089,0.629802,apo25m.5226.150-08-RV.2M03415658+4626067
4183,-0.707683,0.638395,apo25m.5226.150-08-RV.2M03415658+4626067
4184,-0.823701,0.754413,apo25m.5226.150-08-RV.2M03415658+4626067
